# Démonstration de calcul sur les cartes cognitives

## Mise en place

On commence par le chargement des modules et des données

In [51]:
from itertools import islice
from cog_maps import CogMaps, CM_LA_MINE_FILENAME, THESAURUS_FILENAME, WEIGHTS_MAP_FILENAME

thesaurus = CogMaps.load_thesaurus_map(THESAURUS_FILENAME)
weights = CogMaps.load_weights(WEIGHTS_MAP_FILENAME)
les_cartes = CogMaps(CM_LA_MINE_FILENAME)

On va regarder le contenu des 5 premières cartes et garder la toute première sous le coude.

In [52]:
for i, w in islice(les_cartes.cog_maps.items(), 5):
    print(i, w)
une_carte = les_cartes.cog_maps[1]


1 ['pollution', 'inondation', 'boom', 'travail', 'retombée']
2 ['richesse', 'malédiction', 'travail', 'destruction', 'histoire', 'frein', 'blocage', 'coutumier']
4 ['travail', 'pollution', 'plainte']
5 ['argent', 'pollution', 'nickel', 'production', 'nature', 'réparation', 'mer', 'inégalité']
6 ['extraction', 'montagne', 'pollution', 'mineur', 'train', 'diversité', 'évolution']


On va maintenant afficher les pondérations disponibles (il y en a 8) : une application qui à chaque position (ou chaque écart) $n$ fait correspondre un poids dans l'intervalle $[0, \ldots, 1]$

In [53]:
# la liste des pondérations définies
print(weights.keys())
# la pondération pos_3_arith 
weights["pos_3_arith"]

dict_keys(['arithmetique', 'inverse', 'pos_1', 'pos_3', 'pos_6', 'pos_3_arith', 'pos_6_arith', 'exponentielle'])


{1: 1.0,
 2: 0.5,
 3: 0.33,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0}

On va maintenant s'intéresser au thésaurus : une série de 3 applications qui à chaque mot fait correspondre un mot/concept qui le généralise :

- mot énoncé $\to$ concept
- concept    $\to$ mot mère
- mot mère   $\to$ mot grand mère

In [54]:
thesaurus.keys()

dict_keys(['concept', 'mother', 'gd_mother'])

Les niveaux du thésaurus permettent de généraliser les cartes : on peut remplacer chaque mot d'une carte par son concept, puis remplacer par le niveau suivant (mère) et encore celui d'après (grand-mère). C'est la composition ($\circ$) des applications associées à chaque niveau de thésaurus.

In [55]:
une_carte_concept = [thesaurus["concept"][mot] for mot in une_carte]
une_carte_mere = [thesaurus["mother"][mot] for mot in une_carte_concept]
une_carte_gd_mere = [thesaurus["gd_mother"][mot] for mot in une_carte_mere]

# exemple sur la première carte
for quad in zip(une_carte, une_carte_concept, une_carte_mere, une_carte_gd_mere):
    print(" -> ".join(quad))

pollution -> pollution -> pollution -> pollution
inondation -> inondation -> changement climatique -> dégradation environnementale
boom -> boom -> passé -> nouvelle-calédonie
travail -> emploi -> emploi -> travail
retombée -> retombée -> économie du pays -> économie


## Les calculs

Les cartes, les cartes "concepts", les cartes "mères" et les cartes "grands mères" sont toutes des objets du même type, on peut faire les trois calculs principaux :

- l'index qui à chaque mot donne ses occurences
- le calcul du nombre d'occurrence de chaque mot à chaque position
- le calcul du nombre d'occurrences, pondéré selon la position
- le calcul des co-occurrences entre couples de mots
  - ceci donne une matrice symétrique qui sert de base pour le rendu sous forme de graphe ou le clustering

In [59]:
# par exemple "doniambo" apparait 8 fois
les_cartes.index["doniambo"]

[(69, 2),
 (154, 4),
 (187, 4),
 (229, 3),
 (231, 5),
 (248, 4),
 (310, 8),
 (336, 14)]

In [63]:
# sans pondération on voit un poids de 8.0
les_cartes.occurrences["doniambo"]

8.0

In [67]:
# mais si on pondère, le score va être plus bas
les_cartes.weights = weights["pos_3_arith"]
# il apparait une fois en positions 2 et une fois en 3, avec ces poids il va avoir 1/2 + 1/3
les_cartes.occurrences["doniambo"]

0.8300000000000001

In [77]:
# on peut maintenant demander avec qui "doniambo" co-apparait comme mot voisin
les_cartes.weights = weights["pos_1"]
sorted(((poids, mot) for mot, poids in les_cartes.matrix["doniambo"].items() if poids > 0.0), reverse=True)

# on peut vérifier que le nombre total de co-occurences est bien de 16

[(8.0, 'doniambo'),
 (3.0, 'vale'),
 (2.0, 'nickel'),
 (1.0, 'yaté'),
 (1.0, 'vkp'),
 (1.0, 'tiébaghi'),
 (1.0, 'terre rouge'),
 (1.0, 'sln'),
 (1.0, 'métallurgie'),
 (1.0, 'métal'),
 (1.0, 'minerai'),
 (1.0, 'goro'),
 (1.0, 'exportation'),
 (1.0, 'camion')]

In [80]:
# ceci sera différent si on change de système de poids
les_cartes.weights = weights["exponentielle"]
sorted(((poids, mot) for mot, poids in les_cartes.matrix["doniambo"].items() if poids > 0.05), reverse=True)


[(8.0, 'doniambo'),
 (3.0, 'vale'),
 (2.7, 'nickel'),
 (1.7400000000000002, 'sln'),
 (1.3900000000000001, 'métal'),
 (1.37, 'terre rouge'),
 (1.37, 'camion'),
 (1.1400000000000001, 'yaté'),
 (1.1400000000000001, 'goro'),
 (1.1400000000000001, 'exportation'),
 (1.0, 'vkp'),
 (1.0, 'tiébaghi'),
 (1.0, 'métallurgie'),
 (1.0, 'minerai'),
 (0.39, 'emploi'),
 (0.38, 'richesse'),
 (0.37, 'vavouto'),
 (0.37, 'smsp'),
 (0.37, 'poro'),
 (0.37, 'nouvelle-calédonie'),
 (0.37, 'garniérite'),
 (0.37, 'eramet'),
 (0.14, 'sécheresse'),
 (0.14, 'pollution'),
 (0.14, 'koniambo'),
 (0.14, 'investissement'),
 (0.14, 'inox'),
 (0.14, 'côte ouest'),
 (0.14, 'cheminée'),
 (0.07, 'travail')]

## L'exploitation des graphes/matrices

On a un matériel "classique", que l'on peut visualiser.